In [182]:
import pandas as pd
import numpy as np

In [183]:
X_train = pd.read_csv('../data/participants_train.csv')
X_test = pd.read_csv('../data/participants_test.csv')
y_train = pd.read_csv('../data/train_winners.csv')
champ_mastery = pd.read_csv('../data/champion_mastery.csv')

In [184]:
X_train = X_train.merge(
    right = champ_mastery[['summonerId', 'championId', 'championPoints']], 
    how = 'left', 
    on = ['summonerId', 'championId'])

In [185]:
X_train['championPoints'] = X_train['championPoints'].fillna(0)

In [186]:
X_train.head(10)

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championPoints
0,0,100,1,0,303,Mordekaiser,82,61890.0
1,0,100,2,1,616,Sylas,517,37664.0
2,0,100,3,2,667,Lissandra,127,40303.0
3,0,100,4,3,860,Caitlyn,51,96304.0
4,0,100,5,4,325,Morgana,25,608721.0
5,0,200,6,5,459,Warwick,19,1180052.0
6,0,200,7,6,416,LeeSin,64,45215.0
7,0,200,8,7,338,Ahri,103,118831.0
8,0,200,9,8,344,Samira,360,16631.0
9,0,200,10,9,532,Alistar,12,89581.0


X_train['lane'] = np.where(
    (X_train['participantId'] == 1) | (X_train['participantId'] == 6), 'top',
np.where(
    (X_train['participantId'] == 2) | (X_train['participantId'] == 7), 'jungle',
np.where(
    (X_train['participantId'] == 3) | (X_train['participantId'] == 8), 'mid',
np.where(
    (X_train['participantId'] == 4) | (X_train['participantId'] == 9), 'bot', 'support'
))))

In [187]:
matchIds = X_train['matchId'].drop_duplicates().reset_index(drop = True).values.tolist()

In [188]:
df = []

In [189]:
for matchId in matchIds:
    game = X_train.loc[X_train['matchId'] == matchId].reset_index(drop = True)
    
    df.append({
        'matchId' : game['matchId'][0],
#         'top_sumLevel_diff' : (
#             game.loc[game['participantId'] == 1]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 6]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'top_champPoints_diff' : (
            game.loc[game['participantId'] == 1]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 6]['championPoints'].reset_index(drop = True)[0]
        ),
#         'jungle_sumLevel_diff' : (
#             game.loc[game['participantId'] == 2]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 7]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'jungle_champPoints_diff' : (
            game.loc[game['participantId'] == 2]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 7]['championPoints'].reset_index(drop = True)[0]
        ),
#         'mid_sumLevel_diff' : (
#             game.loc[game['participantId'] == 3]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 8]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'mid_champPoints_diff' : (
            game.loc[game['participantId'] == 3]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 8]['championPoints'].reset_index(drop = True)[0]
        ),
#         'bot_sumLevel_diff' : (
#             game.loc[game['participantId'] == 4]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 9]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'bot_champPoints_diff' : (
            game.loc[game['participantId'] == 4]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 9]['championPoints'].reset_index(drop = True)[0]
        ),
#         'support_sumLevel_diff' : (
#             game.loc[game['participantId'] == 5]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 10]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'support_champPoints_diff' : (
            game.loc[game['participantId'] == 5]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 10]['championPoints'].reset_index(drop = True)[0]
        )
    })

In [190]:
X_train = pd.DataFrame(df)

In [191]:
y_train = y_train['winner'] == 100

In [192]:
from sklearn.linear_model import LogisticRegression

In [193]:
logreg = LogisticRegression().fit(X_train.drop(columns = ['matchId']), y_train)

In [194]:
from sklearn.metrics import accuracy_score

In [195]:
accuracy_score(y_train, logreg.predict(X_train.drop(columns = ['matchId'])))

0.527875

In [196]:
X_test = X_test.merge(
    right = champ_mastery[['summonerId', 'championId', 'championPoints']], 
    how = 'left', 
    on = ['summonerId', 'championId'])

In [197]:
X_test['championPoints'] = X_test['championPoints'].fillna(0)

In [198]:
matchIds = X_test['matchId'].drop_duplicates().reset_index(drop = True).values.tolist()

In [199]:
df = []

In [200]:
for matchId in matchIds:
    game = X_test.loc[X_test['matchId'] == matchId].reset_index(drop = True)
    
    df.append({
        'matchId' : game['matchId'][0],
#         'top_sumLevel_diff' : (
#             game.loc[game['participantId'] == 1]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 6]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'top_champPoints_diff' : (
            game.loc[game['participantId'] == 1]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 6]['championPoints'].reset_index(drop = True)[0]
        ),
#         'jungle_sumLevel_diff' : (
#             game.loc[game['participantId'] == 2]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 7]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'jungle_champPoints_diff' : (
            game.loc[game['participantId'] == 2]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 7]['championPoints'].reset_index(drop = True)[0]
        ),
#         'mid_sumLevel_diff' : (
#             game.loc[game['participantId'] == 3]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 8]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'mid_champPoints_diff' : (
            game.loc[game['participantId'] == 3]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 8]['championPoints'].reset_index(drop = True)[0]
        ),
#         'bot_sumLevel_diff' : (
#             game.loc[game['participantId'] == 4]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 9]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'bot_champPoints_diff' : (
            game.loc[game['participantId'] == 4]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 9]['championPoints'].reset_index(drop = True)[0]
        ),
#         'support_sumLevel_diff' : (
#             game.loc[game['participantId'] == 5]['summonerLevel'].reset_index(drop = True)[0] - 
#             game.loc[game['participantId'] == 10]['summonerLevel'].reset_index(drop = True)[0]
#         ),
        'support_champPoints_diff' : (
            game.loc[game['participantId'] == 5]['championPoints'].reset_index(drop = True)[0] - 
            game.loc[game['participantId'] == 10]['championPoints'].reset_index(drop = True)[0]
        )
    })

In [201]:
X_test = pd.DataFrame(df)

In [202]:
logreg.predict(X_test.drop(columns = ['matchId']))

array([ True, False,  True, ..., False, False, False])

In [205]:
submissionV2 = X_test[['matchId']]

In [208]:
submissionV2 = submissionV2.copy()

In [209]:
submissionV2['win'] = logreg.predict(X_test.drop(columns = ['matchId']))

In [212]:
submissionV2['winner'] = np.where(submissionV2['win'] == True, 100, 200)

In [214]:
submissionV2 = submissionV2[['matchId', 'winner']]

In [215]:
submissionV2.to_csv('../submissions/submissionV2.csv', index = False)